In [25]:
import requests
import bs4
from bs4 import BeautifulSoup
import re
import math
import csv
from csv import writer
import concurrent.futures
import random

In [39]:
proxylist =[]
with open('working_proxies.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        proxylist.append(row[0])
print(f'Proxies: {len(proxylist)}')

Proxies: 16


In [104]:
teamID_dict = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHA",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS"
}

In [105]:
#initalize csv
head = ['date', 'homeTeam', 'homeTeam_id', 'homeTeam_points_total', 'homeTeam_points_q1', 'homeTeam_points_q2', 'homeTeam_points_q3', 'homeTeam_points_q4',
        'homeTeam_points_1OT', 'homeTeam_points_2OT', 'homeTeam_points_3OT', 'homeTeam_points_4OT', 
        'homeTeam_FG', 'homeTeam_3P', 'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb', 'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 
        'homeTeam_TO', 'homeTeam_PF', 'awayTeam', 'awayTeam_id', 'awayTeam_points_total', 'awayTeam_points_q1', 'awayTeam_points_q2', 'awayTeam_points_q3', 
        'awayTeam_points_q4', 'awayTeam_points_1OT', 'awayTeam_points_2OT', 'awayTeam_points_3OT', 'awayTeam_points_4OT', 'awayTeam_FG', 'awayTeam_3P', 
        'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb', 'awayTeam_Ast', 'awayTeam_Stl', 'awayTeam_Blk', 'awayTeam_TO', 'awayTeam_PF', 'winner']

with open('nbaDatabase.csv', 'w', encoding='UTF8', newline='') as nbaDatabase:
    writer = csv.writer(nbaDatabase)
    writer.writerow(head)

In [106]:
url = f'https://www.landofbasketball.com/box_scores/2021/1117CHIPOR.htm'
payload = ""
headers = {
    "User-Agent":'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'

}
proxy = random.choice(proxylist)
response = requests.request("GET", url, data=payload, headers=headers, proxies={'http': f"http://{proxy}, 'https:'https/{proxy}"})

print(response)
#soup
soup = BeautifulSoup(response.content, 'html.parser')

#initialize variables
date = None
awayTeam = None
homeTeam = None
awayTeam_id = None
homeTeam_id = None
awayTeam_points_total = None
homeTeam_points_total = None
awayTeam_points_q1 = None
homeTeam_points_q1 = None
awayTeam_points_q2 = None
homeTeam_points_q2 = None
awayTeam_points_q3 = None
homeTeam_points_q3 = None
awayTeam_points_q4 = None
homeTeam_points_q4 = None
awayTeam_points_1OT = 0
homeTeam_points_1OT = 0
awayTeam_points_2OT = 0
homeTeam_points_2OT = 0
awayTeam_points_3OT = 0
homeTeam_points_3OT = 0
awayTeam_points_4OT = 0
homeTeam_points_4OT = 0
awayTeam_FG = None
homeTeam_FG = None
awayTeam_3P = None
homeTeam_3P = None
awayTeam_FT = None
homeTeam_FT = None
awayTeam_O_Reb = None
homeTeam_O_Reb = None
awayTeam_D_Reb = None
homeTeam_D_reb = None
awayTeam_Total_Reb = None
homeTeam_Total_Reb = None
awayTeam_Ast = None
homeTeam_Ast = None
awayTeam_Stl = None
homeTeam_Stl = None
awayTeam_Blk = None
homeTeam_Blk = None
awayTeam_TO = None
homeTeam_TO = None
awayTeam_PF = None
homeTeam_PF = None
overtimes = 0
winner = None

<Response [200]>


In [107]:
overtime_points = {
    'homeTeam_points_1OT': 0,
    'awayTeam_points_1OT': 0,
    'homeTeam_points_2OT': 0,
    'awayTeam_points_2OT': 0,
    'homeTeam_points_3OT': 0,
    'awayTeam_points_3OT': 0,
    'homeTeam_points_4OT': 0,
    'awayTeam_points_4OT': 0
}

In [108]:

#scrape+clean date
dateHome = soup.find('div', class_=re.compile('pad-b10 pad-l5'))
dateHome = dateHome.text.strip().split('\n')
date = dateHome[0].split(': ')
date = date[1]
print(date)


November 17, 2021


In [109]:
#scrape+clean homeTeam
homeTeam = dateHome[1].split(': ')
homeTeam = homeTeam[1]
print(homeTeam)

Portland Trail Blazers


In [110]:
table = soup.find('table', class_=re.compile('size-11 a-left'))
tableText = table.text.strip().split('\n')
for i in range(len(tableText)):
    newI = tableText[i].replace('\t', '')
    tableText[i] = newI

filtered_text = [item for item in tableText if item]
filtered_text.pop(2)
team1 = filtered_text[:2]
team2 = filtered_text[2:]

print(team1)
print(team2)

['107', 'Chicago Bulls']
['112', 'Portland Trail Blazers']


In [111]:
#scrape+clean awayTeam and point totals
if homeTeam in team1:
    homeTeam_points_total = team1[0]
    awayTeam = team2[1]
    awayTeam_points_total = team2[0]
else:
    homeTeam_points_total = team2[0]
    awayTeam = team1[1]
    awayTeam_points_total = team1[0]
print(homeTeam_points_total)
print(awayTeam_points_total)

112
107


In [112]:
if 'OT' in homeTeam_points_total:
    getOT = homeTeam_points_total.split(' ')
    try:
        overtimes = int(getOT[1].replace('(', '').replace(')', '').replace('O', '').replace('T', ''))
    except:
        overtimes = 1
    print(overtimes)
if 'OT' in awayTeam_points_total:
    getOT = awayTeam_points_total.split(' ')
    try:
        overtimes = int(getOT[1].replace('(', '').replace(')', '').replace('O', '').replace('T', ''))
    except:
        overtimes = 1
        

In [113]:
#extract team ids from dict
homeTeam_id = teamID_dict[f'{homeTeam}']
awayTeam_id = teamID_dict[f'{awayTeam}']
print(homeTeam_id)
print(awayTeam_id)

POR
CHI


In [114]:
#scrape + clean points per quarter
quarterTable = soup.find('table', re.compile('by pad-y5 margen-b5'))
quarterStats = quarterTable.find_all('td')
qStatList = []


for stat in quarterStats:
    qStatList.append(stat.text.strip())
team1qStats = qStatList[overtimes+6:overtimes+overtimes+12]
team2qStats = qStatList[overtimes+overtimes+12:]
if homeTeam_id in str(team1qStats[0]):
    homeTeam_points_q1 = team1qStats[1]
    homeTeam_points_q2 = team1qStats[2]
    homeTeam_points_q3 = team1qStats[3]
    homeTeam_points_q4 = team1qStats[4]
    homeTeam_points_total = team1qStats[len(team1qStats)-1]
    awayTeam_points_q1 = team2qStats[1]
    awayTeam_points_q2 = team2qStats[2]
    awayTeam_points_q3 = team2qStats[3]
    awayTeam_points_q4 = team2qStats[4]
    awayTeam_points_total = team2qStats[len(team2qStats)-1]
if awayTeam_id in str(team1qStats[0]):
    homeTeam_points_q1 = team2qStats[1]
    homeTeam_points_q2 = team2qStats[2]
    homeTeam_points_q3 = team2qStats[3]
    homeTeam_points_q4 = team2qStats[4]
    homeTeam_points_total = team2qStats[len(team2qStats)-1]
    awayTeam_points_q1 = team1qStats[1]
    awayTeam_points_q2 = team1qStats[2]
    awayTeam_points_q3 = team1qStats[3]
    awayTeam_points_q4 = team1qStats[4]
    awayTeam_points_total = team1qStats[len(team1qStats)-1]


print(team1qStats)
print(team2qStats)

team1qStats.pop()
team2qStats.pop()

print(team1qStats)
print(team2qStats)

try:
    team1OTStats = team1qStats[5:]
    team2OTStats = team2qStats[5:]

    for i in range(len(team1OTStats)):
        if homeTeam_id in str(team1qStats[0]):
            overtime_points[f'homeTeam_points_{i+1}OT'] = team1OTStats[i]
            overtime_points[f'awayTeam_points_{i+1}OT'] = team2OTStats[i]
        if awayTeam_id in str(team1qStats):
            overtime_points[f'homeTeam_points_{i+1}OT'] = team2OTStats[i]
            overtime_points[f'awayTeam_points_{i+1}OT'] = team1OTStats[i]
except:
    pass

awayTeam_points_1OT = overtime_points['awayTeam_points_1OT']
homeTeam_points_1OT = overtime_points['homeTeam_points_1OT']
awayTeam_points_2OT = overtime_points['awayTeam_points_2OT']
homeTeam_points_2OT = overtime_points['homeTeam_points_2OT']
awayTeam_points_3OT = overtime_points['awayTeam_points_3OT']
homeTeam_points_3OT = overtime_points['homeTeam_points_3OT']
awayTeam_points_4OT = overtime_points['awayTeam_points_4OT']
homeTeam_points_4OT = overtime_points['awayTeam_points_4OT']

print(awayTeam_points_1OT)
print(homeTeam_points_1OT)
print(awayTeam_points_2OT)
print(homeTeam_points_2OT)
print(awayTeam_points_3OT)
print(homeTeam_points_3OT)

['CHI', '33', '30', '22', '22', '107']
['POR', '24', '24', '33', '31', '112']
['CHI', '33', '30', '22', '22']
['POR', '24', '24', '33', '31']
0
0
0
0
0
0


In [101]:
#scrape + clean all other stats
statTables = soup.find_all('div', class_=re.compile('clearfix clear pad-y5'))
for i in range(len(statTables)):
    team1 = statTables[i].find('h3', class_=re.compile('d-inline-block a-middle size-11 margen-t0 margen-b0 negri'))
    if awayTeam in team1.text.strip():
        awayTeamStats = statTables[i].find('tr', class_=re.compile('color-5 negri a-top'))
        awayTeamStats = awayTeamStats.text.strip().split('\n')
        awayTeam_FG = awayTeamStats[3]
        awayTeam_3P = awayTeamStats[4]
        awayTeam_FT = awayTeamStats[5]
        awayTeam_O_Reb = awayTeamStats[6]
        awayTeam_D_Reb = awayTeamStats[7]
        awayTeam_Total_Reb = awayTeamStats[8]
        awayTeam_Ast = awayTeamStats[9]
        awayTeam_Stl = awayTeamStats[10]
        awayTeam_Blk = awayTeamStats[11]
        awayTeam_TO = awayTeamStats[12]
        awayTeam_PF = awayTeamStats[13]
    if homeTeam in team1.text.strip():
        homeTeamStats = statTables[i].find('tr', class_=re.compile('color-5 negri a-top'))
        homeTeamStats = homeTeamStats.text.strip().split('\n')
        homeTeam_FG = homeTeamStats[3]
        homeTeam_3P = homeTeamStats[4]
        homeTeam_FT = homeTeamStats[5]
        homeTeam_O_Reb = homeTeamStats[6]
        homeTeam_D_Reb = homeTeamStats[7]
        homeTeam_Total_Reb = homeTeamStats[8]
        homeTeam_Ast = homeTeamStats[9]
        homeTeam_Stl = homeTeamStats[10]
        homeTeam_Blk = homeTeamStats[11]
        homeTeam_TO = homeTeamStats[12]
        homeTeam_PF = homeTeamStats[13]


In [102]:
#extract winner
home_points = int(re.search(r'\d+', homeTeam_points_total).group())
away_points = int(re.search(r'\d+', awayTeam_points_total).group())
if home_points > away_points:
    winner = homeTeam
elif away_points > home_points:
    winner = awayTeam
else:
    winner = "Draw"

In [103]:
gameStats = [
    [date, homeTeam, homeTeam_id, homeTeam_points_total, homeTeam_points_q1, homeTeam_points_q2, homeTeam_points_q3, homeTeam_points_q4, homeTeam_points_1OT, homeTeam_points_2OT, homeTeam_points_3OT, homeTeam_points_4OT,
    homeTeam_FG, homeTeam_3P, homeTeam_O_Reb, homeTeam_D_Reb, homeTeam_Total_Reb, homeTeam_Ast, homeTeam_Stl, homeTeam_Blk, 
    homeTeam_TO, homeTeam_PF, awayTeam, awayTeam_id, awayTeam_points_total, awayTeam_points_q1, awayTeam_points_q2, awayTeam_points_q3,
    awayTeam_points_q4, awayTeam_points_1OT, awayTeam_points_2OT, awayTeam_points_3OT,
    awayTeam_points_4OT, awayTeam_FG, awayTeam_3P, awayTeam_O_Reb, awayTeam_D_Reb, awayTeam_Total_Reb, awayTeam_Ast, awayTeam_Stl, 
    awayTeam_Blk, awayTeam_TO, awayTeam_PF, winner],
]

with open('nbaDatabase.csv', 'a', encoding='UTF8', newline='') as nbaDatabase:
    writer = csv.writer(nbaDatabase)      
    writer.writerows(gameStats) 